## Staging: Ophyd's Hook for Device-Specific Setup and Cleanup

When bluesky obtains a reading from some `device` it typically:

* Calls `device.trigger()` and receives back a status object
* Waits for that status object to complete (while potentially doing other things, like triggering other detectors in parallel)
* Calls `device.read()`

If it obtains multiple readings in sequence, it repeats this trigger/wait/read cycle. Sometimes, before triggering, there is some choreographed sequence of steps necessary to make `device` ready for use and some corresponding sequence to put in back safely into a resting state. To support this bluesky plans typically call `device.stage()` *once* before first using a device in a plan and then `device.unstage()` at the end. Even if a plan is interrupted by the user or by an exception begin raise, `device.unstage()` will be called.

Like `device.trigger()`, is hook is optional and does not apply to all devices.

It is typically used with detectors, but it can be used with any Device.

In [ ]:
from ophyd import set_and_wait

class TriggeredDetector(Device):
    """
    A detector that requires triggering
    """
    gain = Component(EpicsSignal, ':gain', kind='config')
    exposure_time = Component(EpicsSignal, ':exposure_time', kind='config')
    reading = Component(EpicsSignalRO, ':reading', kind='normal')
    acquire = Component(EpicsSignal, ':acquire', kind='omitted', put_complete=True)
    enabled = Component(EpicsSignal, ':enabled', kind='omitted')

    def trigger(self):
        """
        Trigger the detector and return a Status object.
        """
        status = DeviceStatus(self)
        self.acquire.put(1, callback=status._finished)
        return status
    
    def stage(self):
        self.initial_enabled_state = self.enabled.get()
        set_and_wait(self.enabled, 1)
        return super().stage()
    
    def unstage(self):
        ret =  super().unstage()
        set_and_wait(self.enabled, self.initial_enabled_state)
        return ret
    
triggered_detector = TriggeredDetector('trigger_with_pc', name='triggered_detector')

In [ ]:
triggered_detector.enabled.put(0)
triggered_detector.enabled.get()

In [ ]:
triggered_detector.stage()

In [ ]:
triggered_detector.enabled.get()

In [ ]:
status = triggered_detector.trigger()

In [ ]:
while not status.done:
    time.sleep(0.01)

In [ ]:
triggered_detector.read()

In [ ]:
status = triggered_detector.trigger()

In [ ]:
while not status.done:
    time.sleep(0.01)

In [ ]:
triggered_detector.read()

In [ ]:
triggered_detector.unstage()

In [ ]:
triggered_detector.enabled.get()

### A convenient shorthand for common simple cases: `stage_sigs`

In [ ]:
from ophyd import set_and_wait

class TriggeredDetector(Device):
    """
    A detector that requires triggering
    """
    gain = Component(EpicsSignal, ':gain', kind='config')
    exposure_time = Component(EpicsSignal, ':exposure_time', kind='config')
    reading = Component(EpicsSignalRO, ':reading', kind='hinted')
    acquire = Component(EpicsSignal, ':acquire', kind='omitted', put_complete=True)
    enabled = Component(EpicsSignal, ':enabled', kind='omitted')
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.stage_sigs['enabled'] = 1  # OrderedDict mapping component name to desired state

    def trigger(self):
        """
        Trigger the detector and return a Status object.
        """
        status = DeviceStatus(self)
        self.acquire.put(1, callback=status._finished)
        return status
    
triggered_detector = TriggeredDetector('trigger_with_pc', name='triggered_detector')

## Exercise

Try staging the device twice in a row. Then try unstaging it twice in a row.

## Customizing cleanup via `stop`, `resume`, `pause`

These optional methods can be used to further customize a Device's cleanup:

* `stop` -- called by bluesky when a plan is paused or exits (successfully or in error)
* `pause` -- called when the RunEngine is paused
* `resume` -- called when the RunEngine resumes from a pause

In [ ]:
class TriggeredDetector(Device):
    """
    A detector that requires triggering
    """

In [ ]:
    gain = Component(EpicsSignal, ':gain', kind='config')
    exposure_time = Component(EpicsSignal, ':exposure_time', kind='config')
    reading = Component(EpicsSignalRO, ':reading', kind='hinted')
    acquire = Component(EpicsSignal, ':acquire', kind='omitted', put_complete=True)
    enabled = Component(EpicsSignal, ':enabled', kind='omitted')
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.stage_sigs['enabled'] = 1  # OrderedDict mapping component name to desired state

    def trigger(self):
        """
        Trigger the detector and return a Status object.
        """
        status = DeviceStatus(self)
        self.acquire.put(1, callback=status._finished)
        return status
    
    def resume(self):
        ...
        
    def pause(self):
        ...
        
    def stop(self, success=False):
        ...